# Individual Algo Strong Scaling (input types on same plots)

In [1]:
import sys
sys.path.append("/scratch/group/csce435-f23/python-3.8.17/lib/python3.8/site-packages")
sys.path.append("/scratch/group/csce435-f23/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read in all the files

In [2]:
tk = th.Thicket.from_caliperreader(glob("cali_files/*-bubble.cali"))
tk.dataframe = tk.dataframe.drop(["nid", "spot.channel", "Total time"], axis=1)
#tk.dataframe = tk.dataframe.drop(["spot.channel", "Total time"], axis=1)

IndexError: list index out of range

In [ ]:
# Performance data
tk.dataframe

Groupby matrix size argument, available in the metadata (tk.metadata) to organize our performance data into 3 groupings.

In [ ]:
tk.statsframe.dataframe["time"]= 1
print(tk.tree())

In [ ]:
tk.metadata

In [ ]:
gb = tk.groupby("InputSize")
print(gb.keys())
allthickets = list(gb.values())
input65536 = gb[65536].groupby("InputType")
input262144 = gb[262144].groupby("InputType")
input1048576 = gb[1048576].groupby("InputType")
input4194304 = gb[4194304].groupby("InputType")
input16777216 = gb[16777216].groupby("InputType")
input67108864 = gb[67108864].groupby("InputType")
input268435456 = gb[268435456].groupby("InputType")

# inputsorted = gb['sorted'].groupby("InputType")
# inputreversed = gb['reversed'].groupby("InputSize")
# inputperturbed = gb['1 perturbed'].groupby("InputSize")
# inputrandom = gb['random'].groupby("InputSize")

Compose these 3 groups column-wise to get a holistic view of the data

In [ ]:
ctk65536 = th.Thicket.concat_thickets(
    thickets=list(input65536.values()),
    headers=list(input65536.keys()),
    axis="columns",
    metadata_key="num_procs"
)

ctk262144 = th.Thicket.concat_thickets(
    thickets=list(input262144.values()),
    headers=list(input262144.keys()),
    axis="columns",
    metadata_key="num_procs"
)

ctk1048576 = th.Thicket.concat_thickets(
    thickets=list(input1048576.values()),
    headers=list(input1048576.keys()),
    axis="columns",
    metadata_key="num_procs"
)

ctk4194304 = th.Thicket.concat_thickets(
    thickets=list(input4194304.values()),
    headers=list(input4194304.keys()),
    axis="columns",
    metadata_key="num_procs"
)
ctk16777216 = th.Thicket.concat_thickets(
    thickets=list(input16777216.values()),
    headers=list(input16777216.keys()),
    axis="columns",
    metadata_key="num_procs"
)
ctk67108864 = th.Thicket.concat_thickets(
    thickets=list(input67108864.values()),
    headers=list(input67108864.keys()),
    axis="columns",
    metadata_key="num_procs"
)
ctk268435456 = th.Thicket.concat_thickets(
    thickets=list(input268435456.values()),
    headers=list(input268435456.keys()),
    axis="columns",
    metadata_key="num_procs"
)



allctk = []
allctk.append(ctk65536)
allctk.append(ctk262144)
allctk.append(ctk1048576)
allctk.append(ctk4194304)
allctk.append(ctk16777216)
allctk.append(ctk67108864)
allctk.append(ctk268435456)

In [ ]:
for ctk in allctk:
    display(ctk.dataframe)

Perform pandas operations to get the table in a desirable format for plotting

In [ ]:
for ctk in allctk:
    # Drop unwanted columns
    columns_to_drop = [("node", ""), ("index", ""),]
    ctk.dataframe = ctk.dataframe.reset_index().drop(columns_to_drop, axis=1, errors='ignore')
    # Rename index to remove tuple
    ctk.dataframe = ctk.dataframe.rename({("name", ""): "name", ("num_procs", ""): "num_procs"}).set_index(["name", "num_procs"])

In [ ]:
for ctk in allctk:
    display(ctk.dataframe)

In [ ]:
# Define common variables
threads = sorted(list(set(allctk[0].dataframe.index.get_level_values("num_procs"))))

In [ ]:
# Drop unwanted columns
for ctk in allctk:
    columns_to_drop = [('1 perturbed', 'Min time/rank'), ('1 perturbed', 'Max time/rank'),
                       ('random', 'Max time/rank'), ('random', 'Min time/rank'),
                       ('reversed', 'Max time/rank'), ('reversed', 'Min time/rank'),
                       ('sorted', 'Max time/rank'), ('sorted', 'Min time/rank')]
    ctk.dataframe = ctk.dataframe.drop(columns_to_drop, axis=1, errors='ignore')
    function_names = sorted(ctk.dataframe.columns.tolist())
print(ctk.dataframe.columns)
print(function_names)


In [ ]:
for ctk in allctk:
    display(ctk.metadata)

# Plotting

### Option 1
Use `pandas.DataFrame.plot` to automatically interface with matplotlib

### Option 2
Use `matplotlib.pyplot` directly for more fine-grained control. Specifically for our case we can change the x axis scaling from base 10 to base 2.

In [ ]:
def plot_mpl(df, title, xlabel, ylabel):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o',
    )
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend(df.columns)
    plt.grid(True)
    #plt.yscale("log", base=10)
    plt.xscale("log", base=2)
    plt.xticks(threads)
    plt.show()
    return fig, ax

In [ ]:
def plot_mpl3(df,df1, df2, title, xlabel, ylabel, label1, label2, label3):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o', label=label1
    )
    ax.plot(df1, marker='>', label=label2)
    ax.plot(df2, marker='<', label=label3)
    ax.plot()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    plt.grid(True)
    #plt.yscale("log", base=10)
    plt.xscale("log", base=2)
    plt.xticks(threads)
    plt.show()
    return fig, ax

In [ ]:
def plot_mpl2(df,df1, title, xlabel, ylabel, label1, label2):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o', label=label1
    )
    ax.plot(df1, marker='>', label=label2)
    ax.plot()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    plt.grid(True)
    #plt.yscale("log", base=10)
    plt.xscale("log", base=2)
    plt.xticks(threads)
    plt.show()
    return fig, ax

In [ ]:
def plot_mpl_inputsizes(df,df1, df2, df3, df4, df5, df6, title, xlabel, ylabel, label1, label2, label3, label4, label5, label6, label7):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o', label=label1
    )
    ax.plot(df1, marker='>', label=label2)
    ax.plot(df2, marker='^', label=label3)
    ax.plot(df3, marker='o', label=label4)
    ax.plot(df4, marker='v', label=label5)
    ax.plot(df5, marker='1', label=label6)
    ax.plot(df6, marker='2', label=label7)
    ax.plot()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    plt.grid(True)
    #plt.yscale("log", base=10)
    plt.xscale("log", base=2)
    plt.xticks(threads)
    plt.show()
    return fig, ax

In [ ]:
def plot_mpl_inputtypes(df,df1, df2, df3, title, xlabel, ylabel, label1, label2, label3, label4):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(
        df,
        marker='o', label=label1
    )
    ax.plot(df1, marker='>', label=label2)
    ax.plot(df2, marker='<', label=label3)
    ax.plot(df3, marker='v', label=label4)
    ax.plot()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend(loc="upper left", bbox_to_anchor=(1,1))
    plt.grid(True)
    #plt.yscale("log", base=10)
    plt.xscale("log", base=2)
    plt.xticks(threads)
    plt.show()
    return fig, ax

# Examples

## time

In [ ]:
sizes = [ 65536, 262144, 1048576, 4194304, 16777216, 67108864, 268435456]
sorttypes = [('1 perturbed', 'Avg time/rank'),
            (     'random', 'Avg time/rank'),
            (   'reversed', 'Avg time/rank'),
            (     'sorted', 'Avg time/rank')]
# need to change sizes to be sizes then create an mpl function to take in a df_ of every size
print(ctk.dataframe.columns)

In [ ]:
regions = sorted(list(set(ctk.dataframe.index.get_level_values("name"))))
print(regions)

for ctk in allctk:
    print(ctk.dataframe)

In [ ]:
i = 0
for ctk in allctk:
    plot_mpl_inputtypes(
        df=ctk.dataframe[[sorttypes[0]]].loc["main"],
        df1= ctk.dataframe[[sorttypes[1]]].loc["main"],
        df2= ctk.dataframe[[sorttypes[2]]].loc["main"],
        df3= ctk.dataframe[[sorttypes[3]]].loc["main"],
        title=f"Strong Scaling {sizes[i]}: Main",
        xlabel="Procs",
        ylabel="Time (seconds)",
        label1='1 perturbed',
        label2='random',
        label3='reversed',
        label4= 'sorted'
    )
    i += 1

# Plot of 2

In [ ]:
# i = 0
# for ctk in allctk:
#     plot_mpl2(
#         df= ctk.dataframe[[temp[1]]].loc["main"],
#         df1= ctk.dataframe[[temp[2]]].loc["main"],
#         title=f"Sorted Weak Scaling {sizes[i]}: Main",
#         xlabel="Threads",
#         ylabel="Time (seconds)",
#         label1='Sample',
#         label2='Merge'
#     )
#     i += 1

## Comp_Large Graphs (for 3 then 2)

In [ ]:
i = 0
for ctk in allctk:
    plot_mpl_inputtypes(
        df=ctk.dataframe[[sorttypes[0]]].loc["comp_large"],
        df1= ctk.dataframe[[sorttypes[1]]].loc["comp_large"],
        df2= ctk.dataframe[[sorttypes[2]]].loc["comp_large"],
        df3= ctk.dataframe[[sorttypes[3]]].loc["comp_large"],
        title=f"Strong Scaling {sizes[i]}: Comp Large",
        xlabel="Procs",
        ylabel="Time (seconds)",
        label1='1 perturbed',
        label2='random',
        label3='reversed',
        label4= 'sorted'
    )
    i += 1

## Comm Graphs (for 3 then 2)

In [ ]:
i = 0
for ctk in allctk:
    plot_mpl_inputtypes(
        df=ctk.dataframe[[sorttypes[0]]].loc["comm"],
        df1= ctk.dataframe[[sorttypes[1]]].loc["comm"],
        df2= ctk.dataframe[[sorttypes[2]]].loc["comm"],
        df3= ctk.dataframe[[sorttypes[3]]].loc["comm"],
        title=f"Strong Scaling {sizes[i]}: Comm",
        xlabel="Procs",
        ylabel="Time (seconds)",
        label1='1 perturbed',
        label2='random',
        label3='reversed',
        label4= 'sorted'
    )
    i += 1